## Two Tower Model 학습 시키기

### 1. Feature 테이블에서 데이터 읽어와서 train/test set으로 split 하기

In [61]:
import mysql.connector as sql

import pandas as pd

host = '127.0.0.1'
port = 3306
username = 'root'
password = '1111'
database_name = 'dealicious'

db_connection = sql.connect(host=host, database=database_name, user=username, password=password)

retrievals_df = pd.read_sql('SELECT * FROM rec_retrievals', con=db_connection)
retrievals_df.head(3)

/var/folders/fx/b8s33_bs1nx_2nnz6h7w64qm0000gn/T/ipykernel_95006/2087775427.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  retrievals_df = pd.read_sql('SELECT * FROM rec_retrievals', con=db_connection)


,t_dat,customer_id,article_id,price,sales_channel_id,month_sin,month_cos,age,garment_group_name,index_group_name
0,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,710390001,0.019051,1,-0.866025,-0.5,20.0,Skirts,Divided
1,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,633130019,0.016932,1,-0.866025,-0.5,20.0,Jersey Fancy,Divided
2,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,671057002,0.008458,1,-0.866025,-0.5,20.0,Jersey Fancy,Divided


In [62]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(retrievals_df, test_size=0.2, random_state=42)

In [63]:
train_df["article_id"] = train_df["article_id"].astype(str)  # to be removed
test_df["article_id"] = test_df["article_id"].astype(str)  # to be removed

pandas dataframe 을 tensorflow dataset 으로 변환하기

query embedding에 사용할 feature
- `customer_id`: ID of the customer.
- `age`: age of the customer at the time of purchase.
- `month_sin`, `month_cos`: time of year the purchase was made.

candidate embedding에 사용할 feature
- `article_id`: ID of the item.
- `garment_group_name`: type of garment.
- `index_group_name`: menswear/ladieswear etc.

In [64]:
import tensorflow as tf

query_features = ["customer_id", "age", "month_sin", "month_cos"]
candidate_features = ["article_id", "garment_group_name", "index_group_name"]

def df_to_ds(df):
    return tf.data.Dataset.from_tensor_slices({col : df[col] for col in df})

BATCH_SIZE = 448
train_ds = df_to_ds(train_df).batch(BATCH_SIZE).cache().shuffle(BATCH_SIZE*10)
val_ds = df_to_ds(test_df).batch(BATCH_SIZE).cache()

In [69]:
train_ds.first

AttributeError: '_ShuffleDataset' object has no attribute 'first'

In [6]:
user_id_list = train_df["customer_id"].unique().tolist()
item_id_list = train_df["article_id"].unique().tolist()

garment_group_list = train_df["garment_group_name"].unique().tolist()
index_group_list = train_df["index_group_name"].unique().tolist()

print(f"Number of transactions: {len(train_df):,}")
print(f"Number of users: {len(user_id_list):,}")
print(f"Number of items: {len(item_id_list):,}")
print(garment_group_list)

Number of transactions: 14,784
Number of users: 3,845
Number of items: 6,487
['Jersey Fancy', 'Jersey Basic', 'Accessories', 'Knitwear', 'Trousers Denim', 'Socks and Tights', 'Special Offers', 'Under-, Nightwear', 'Blouses', 'Skirts', 'Trousers', 'Unknown', 'Swimwear', 'Shoes', 'Outdoor', 'Dresses/Skirts girls', 'Dressed', 'Dresses Ladies', 'Woven/Jersey/Knitted mix Baby', 'Shirts', 'Shorts']


### 2. Two Tower Model 학습

다음의 두 모델을 two tower 학습 기법을 통해 학습 시킴.

- `query model` : user, transaction feature를 나타내는 query embedding 생성하는 모델
- `candidate model` : item feature를 나타내는 candidate embedding을 생성하는 모델

In [7]:
EMB_DIM = 16

In [8]:
from models.query_tower import QueryTower

query_model = QueryTower(user_id_list=user_id_list, emb_dim=EMB_DIM)
query_model.normalized_age.adapt(train_ds.map(lambda x : x["age"]))

# Initialize model with inputs.
query_df = train_df[query_features]
query_ds = df_to_ds(query_df).batch(1)
query_model(next(iter(query_ds)))


2024-05-07 16:15:06.365070: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype string and shape [14784]
	 [[{{node Placeholder/_4}}]]
2024-05-07 16:15:06.366394: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [14784]
	 [[{{node Placeholder/_0}}]]
2024-05-07 16:15:06.724193: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype double and shape [14

<tf.Tensor: shape=(1, 16), dtype=float32, numpy=
array([[ 0.28267318,  0.17895192, -0.05836855, -0.03838781,  0.23421174,
         0.6082657 ,  0.23707908, -0.14201935,  0.4328758 , -0.28465068,
         0.3968409 ,  0.01230739,  0.39183363,  0.29431206, -0.04555143,
         0.33463132]], dtype=float32)>

In [9]:
from models.item_tower import ItemTower


item_model = ItemTower(item_id_list=item_id_list, garment_group_list=garment_group_list, index_group_list=index_group_list, emb_dim=EMB_DIM)

item_df = train_df[candidate_features]
item_df.drop_duplicates(subset="article_id", inplace=True)
item_ds = df_to_ds(item_df)

/var/folders/fx/b8s33_bs1nx_2nnz6h7w64qm0000gn/T/ipykernel_95006/2937662747.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_df.drop_duplicates(subset="article_id", inplace=True)


In [10]:
import tensorflow_addons as tfa
from models.two_tower_model import TwoTowerModel

model = TwoTowerModel(query_model=query_model, item_model=item_model, item_ds=item_ds)
optimizer = tfa.optimizers.AdamW(0.001, learning_rate=0.01)
model.compile(optimizer=optimizer)

/Users/jimin/study/two-tower/.venv/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [11]:
model.fit(train_ds, validation_data=val_ds, epochs=5)

Epoch 1/5


2024-05-07 16:15:26.639151: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype double and shape [14784]
	 [[{{node Placeholder/_5}}]]
2024-05-07 16:15:26.639775: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype double and shape [14784]
	 [[{{node Placeholder/_6}}]]


32/33 [============================>.] - ETA: 0s - loss: 2687.4696 - regularization_loss: 0.0000e+00 - total_loss: 2687.4696

2024-05-07 16:15:28.706437: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype double and shape [3697]
	 [[{{node Placeholder/_7}}]]


33/33 [==============================] - 2s 26ms/step - loss: 2683.9153 - regularization_loss: 0.0000e+00 - total_loss: 2683.9153 - val_factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - val_loss: 503.1334 - val_regularization_loss: 0.0000e+00 - val_total_loss: 503.1334
Epoch 2/5
33/33 [==============================] - 1s 17ms/step - loss: 2420.5836 - regularization_loss: 0.0000e+00 - total_loss: 2420.5836 - val_factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - val_loss: 492.

### 3. 모델 저장하기

In [12]:
class QueryModelModule(tf.Module):
    def __init__(self, query_model):
        self.query_model = query_model

    @tf.function()
    def compute_emb(self, instances):
        query_emb = self.query_model(instances)
        return {"customer_id": instances["customer_id"],
                "month_sin": instances["month_sin"],
                "month_cos": instances["month_cos"],
                "query_emb": query_emb}

# wrap query_model:   query_model -> query_model_module
query_model = QueryModelModule(model.query_model)

In [13]:
# 로컬에 query_model 저장
instances_spec={
    'customer_id': tf.TensorSpec(shape=(None,), dtype=tf.string, name='customer_id'),
    'month_sin': tf.TensorSpec(shape=(None,), dtype=tf.float64, name='month_sin'),
    'month_cos': tf.TensorSpec(shape=(None,), dtype=tf.float64, name='month_cos'),
    'age': tf.TensorSpec(shape=(None,), dtype=tf.float64, name='age')
}
signatures = query_model.compute_emb.get_concrete_function(instances_spec)

tf.saved_model.save(query_model, "query_model", signatures=signatures)

INFO:tensorflow:Assets written to: query_model/assets


In [ ]:
# 로컬에 candidate_model 저장
# tf.saved_model.save(model.item_model, "candidate_model")

In [70]:
import boto3

AWS_PROFILE = 'default'
BUCKET_NAME = 'jimin-model'
session = boto3.Session(profile_name=AWS_PROFILE)
s3_client = session.client('s3')

In [14]:
from utils.save_model import SaveModel

instances_spec={
    'customer_id': tf.TensorSpec(shape=(None,), dtype=tf.string, name='customer_id'),
    'month_sin': tf.TensorSpec(shape=(None,), dtype=tf.float64, name='month_sin'),
    'month_cos': tf.TensorSpec(shape=(None,), dtype=tf.float64, name='month_cos'),
    'age': tf.TensorSpec(shape=(None,), dtype=tf.float64, name='age')
}
signatures = query_model.compute_emb.get_concrete_function(instances_spec)

save_model = SaveModel(s3_client, BUCKET_NAME)
save_model.call(query_model, "query", signatures=signatures)
save_model.call(model.item_model, "candidate_model")

INFO:tensorflow:Assets written to: temp/query/1/assets
Uploaded temp/query/1.tar.gz to s3://jimin-model/query/1.tar.gz
INFO:tensorflow:Assets written to: temp/candidate_model/1/assets
Uploaded temp/candidate_model/1.tar.gz to s3://jimin-model/candidate_model/1.tar.gz


### 4. 모델 로드 & 테스트 쿼리

In [71]:
from utils.load_model import LoadModel
import tensorflow as tf

load_model = LoadModel(s3_client, BUCKET_NAME)
loaded_model = load_model.call("query/1")

In [74]:
import json
r = loaded_model.compute_emb({"customer_id": ["1"], "month_sin": [0.5], "month_cos": [0.5], "age": [30.0]})
json.dumps(r)

TypeError: Object of type EagerTensor is not JSON serializable

In [18]:

# loaded_model = tf.saved_model.load("temp/query_model")

# 예측 함수 추출
infer = loaded_model.signatures["serving_default"]

# 입력 스펙 확인
# print(infer.structured_input_signature)

# 샘플 입력 데이터 생성 (double 타입으로)
sample_input = {
    'customer_id': tf.constant(['customer1', 'customer2']),
    'month_sin': tf.constant([0.1, 0.2], dtype=tf.float64),
    'month_cos': tf.constant([0.2, 0.3], dtype=tf.float64),
    'age': tf.constant([30.0, 40.0], dtype=tf.float64)
}

# 예측 수행
predictions = infer(**sample_input)

# 예측 결과 출력
# print(predictions)
print(sample_input)

{'customer_id': <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'customer1', b'customer2'], dtype=object)>, 'month_sin': <tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.1, 0.2])>, 'month_cos': <tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.2, 0.3])>, 'age': <tf.Tensor: shape=(2,), dtype=float64, numpy=array([30., 40.])>}
